In [ ]:
!pip install torch_optimizer

import numpy as np
import random
import math
import time
import matplotlib.pyplot as plt

import torch
import torch.nn as nn

from torch.optim import Optimizer
import torch_optimizer as optim

from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

from torchvision import datasets
from torchvision.transforms import transforms

import torch.nn.functional as F


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
name = torch.cuda.get_device_name(0)
print("GPU: " + name)

random_seed = 0

torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

np.random.seed(random_seed)
random.seed(random_seed)


class EarlyStopping:
    def __init__(self, patience=10, path='checkpoint.pt'):
        
        self.patience = patience
        self.counter = 0
        self.best_score = None
        self.val_loss_min = np.Inf
        self.early_stop = False

        self.path = path



    def __call__(self, val_loss, model):

        score = val_loss

        if self.best_score is None:
            self.best_score = score
            
            print('Validation loss decreased ({:.4f} --> {:.4f}).  Saving model ...'.format(self.val_loss_min, val_loss))
            torch.save(model.state_dict(), self.path)
            self.val_loss_min = val_loss

        elif score > self.best_score:
            self.counter += 1
            print("EarlyStopping counter: {} out of {}".format(self.counter, self.patience))

            if self.counter >= self.patience:
                self.early_stop = True

        else:
            self.best_score = score
            self.counter = 0

            print('Validation loss decreased --- Saving model ...')
            torch.save(model.state_dict(), self.path)
            self.val_loss_min = val_loss



def get_data_len_index(pad=4, randomcrop=32):
    data_shuffle = []

    transform = transforms.Compose([
                                    transforms.Pad(pad),
                                    transforms.RandomHorizontalFlip(),
                                    transforms.RandomCrop(randomcrop),
                                    transforms.ToTensor()
    ])

    train_dataset = datasets.CIFAR10(root='./cifar_10data/',
                                     train=True,
                                     transform=transform,
                                     download=True) 
    
    len_train = len(train_dataset)
    index_train = list(range(len_train))
    

    data_shuffle.append(len_train)
    data_shuffle.append(index_train)
    data_shuffle.append(train_dataset)

    return data_shuffle



def test_model(model, batch_size=128):
    model.eval()
    test_loss, correct, total = 0, 0, 0

    with torch.no_grad():
        for images, labels in test_loader :
            images, labels = images.to(device), labels.to(device)

            output = model(images)
            test_loss += loss_function(output, labels).item()

            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(labels.view_as(pred)).sum().item()

            total += labels.size(0)

    print('[Test set] Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
            test_loss /total, correct, total,
            100. * correct / total))
    

def train_model(model, batch_size, n_epochs, patience, loader):
    train_loader = loader[0]
    valid_loader = loader[1]
    
    train_losses = []
    valid_losses = []

    avg_train_losses = []
    avg_valid_losses = []

    early_stopping = EarlyStopping(patience=patience)

    model.train()

    start = time.time()

    for epoch in range(1,n_epochs+1):
        print("{}th Epoch starting.".format(epoch))

        for i, (images, labels) in enumerate(train_loader):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()

            output = model(images)

            train_loss = loss_function(output, labels)

            train_loss.backward()

            optimizer.step()

            loss = train_loss.item()


            train_losses.append(loss)


        model.eval()

        with torch.no_grad():
            for images, labels in valid_loader:
                images , labels = images.to(device), labels.to(device)

                output = model(images)

                valid_loss = loss_function(output, labels)

                loss = valid_loss.item()

                valid_losses.append(loss)
        
        loss_train = np.average(train_losses)
        loss_valid = np.average(valid_losses)

        avg_train_losses.append(loss_train)
        avg_valid_losses.append(loss_valid)

        print("Epoch [{}] Train Loss: {:.4f} & Validation Loss: {:.4f}".format(epoch, loss_train, loss_valid))

        train_losses = []
        valid_losses = []

        early_stopping(loss_valid, model)

        if early_stopping.early_stop:
            print("Early Stopping!!")
            end = time.time()
            #print(end-start)
            break

    model.load_state_dict(torch.load('checkpoint.pt'))

    return model, avg_train_losses, avg_valid_losses



def train_KFold(model, batch_size, n_epochs, patience, data_info, fold):
        
    train_losses = []
    valid_losses = []

    loader = []

    path='checkpoint.pt'

    len_train = data_info[0]
    index_train = data_info[1]
    train_dataset = data_info[2]

    np.random.shuffle(index_train)

    split_size = len_train // fold

    start = time.time()
    for i in range(fold):

        valid_ind = index_train[split_size * i: split_size * (i+1)]
                
        if i == 0:
            train_ind = index_train[split_size * (i+1):]
                    
        elif i == 4:
            train_ind = index_train[:split_size*i]

        else:
            train_ind = index_train[:split_size * i] + index_train[split_size * (i+1):]

        train_sampler = SubsetRandomSampler(train_ind)
        valid_sampler = SubsetRandomSampler(valid_ind)    

        train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                                        batch_size=batch_size,
                                                        sampler = train_sampler,
                                                        num_workers=0)
                
        valid_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                                        batch_size=batch_size,
                                                        sampler = valid_sampler,
                                                        num_workers=0)
        
        loader.append(train_loader)
        loader.append(valid_loader)
                

        print("{} Fold is Training".format(i+1))

        model, avg_train_losses, avg_valid_losses = train_model(model, batch_size, n_epochs, patience, loader)
        
        train_losses.append(avg_train_losses)
        valid_losses.append(avg_valid_losses)
        
    end = time.time()

    print(end-start)

    return model, train_losses, valid_losses


     |████████████████████████████████| 71kB 8.4MB/s 
cuda:0
GPU: Tesla T4


In [ ]:
import torch.nn.init as init

def weight_init(m):
  if isinstance(m, nn.Conv2d):
    init.kaiming_uniform_(m.weight.data)

### He Uniform Initialization

### Batch_size 63 --> 256  
### Weight_decay 5e-4 --> 5e-5


In [ ]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

In [ ]:
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

In [ ]:
def ResNet34():
    return ResNet(BasicBlock, [3, 4, 6, 3])

In [ ]:
data_info = get_data_len_index()

Extracting ./cifar_10data/cifar-10-python.tar.gz to ./cifar_10data/


In [ ]:
model = ResNet34().to(device)
loss_function = torch.nn.CrossEntropyLoss()
optimizer = optim.RAdam(model.parameters(), weight_decay=5e-5)
batch_size = 256

In [ ]:
model, train_losses, valid_losses = train_KFold(model, batch_size=batch_size, n_epochs=250, patience=20, data_info=data_info, fold=5)

1 Fold is Training
1th Epoch starting.
Epoch [1] Train Loss: 1.6782 & Validation Loss: 1.5393
Validation loss decreased (inf --> 1.5393).  Saving model ...
2th Epoch starting.
Epoch [2] Train Loss: 2.2616 & Validation Loss: 1.9003
EarlyStopping counter: 1 out of 20
3th Epoch starting.
Epoch [3] Train Loss: 1.8087 & Validation Loss: 1.7220
EarlyStopping counter: 2 out of 20
4th Epoch starting.
Epoch [4] Train Loss: 1.6918 & Validation Loss: 1.6073
EarlyStopping counter: 3 out of 20
5th Epoch starting.
Epoch [5] Train Loss: 1.5836 & Validation Loss: 1.5316
Validation loss decreased --- Saving model ...
6th Epoch starting.
Epoch [6] Train Loss: 1.4910 & Validation Loss: 1.4638
Validation loss decreased --- Saving model ...
7th Epoch starting.
Epoch [7] Train Loss: 1.4099 & Validation Loss: 1.3713
Validation loss decreased --- Saving model ...
8th Epoch starting.
Epoch [8] Train Loss: 1.3209 & Validation Loss: 1.2982
Validation loss decreased --- Saving model ...
9th Epoch starting.
Epoch 

In [ ]:
 test_dataset = datasets.CIFAR10(root='./cifar_10data/',
                                train=False,
                                transform=transforms.ToTensor(),
                                download=True) 

test_loader = torch.utils.data.DataLoader(test_dataset,
                                          batch_size=batch_size,
                                          num_workers=0)

Files already downloaded and verified


In [ ]:
test_model(model)

[Test set] Average loss: 0.0016, Accuracy: 8853/10000 (88.53%)

